In [2]:
# Install dependencies
!pip install -q langchain langchain-community faiss-cpu sentence-transformers pypdf



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 9.4 MB/s eta 0:00:00


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import os

In [10]:
# Load the Constitution PDF
pdf_path = "/content/drive/MyDrive/The_Constitution_of_Kenya_2010.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [9]:
#  Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(documents)

In [ ]:
#  Create free embeddings (using MiniLM)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-3253230758.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [11]:
#  Store in FAISS vector database
vectorstore = FAISS.from_documents(docs, embeddings)

In [12]:
os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    api_key=os.environ["GROQ_API_KEY"]
)

In [13]:
#  Build RAG pipeline
qa = RetrievalQA.from_chain_type(
    llm=llm,#uses Groq’s LLM to combine the chunks + user query into a final answer.
    retriever=vectorstore.as_retriever(search_kwargs={"k":3}),#pulls the top 3 relevant Constitution chunks.
    chain_type="stuff"
)

In [14]:
#  Test it
query = "What does the Constitution say about the right to life?"
answer = qa.run(query)
print("Q:", query)
print("A:", answer)

/tmp/ipython-input-3661509434.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(query)


Q: What does the Constitution say about the right to life?
A: According to the Constitution of Kenya, 2010, the right to life is stated in Article 26. It says:

"(1) Every person has the right to life.
(2) The life of a person begins at conception.
(3) A person shall not be deprived of life intentionally, except to the extent authorised by this Constitution or other written law.
(4) Abortion is not permitted unless, in the opinion of a trained health professional, there is need for emergency treatment, or the life or health of the mother is in danger, or if permitted by any other written law."

This article establishes that every person has the right to life, and that life begins at conception. It also sets out the circumstances under which a person's life can be intentionally deprived, which are limited to situations authorised by the Constitution or other written law.
